## imports

In [7]:

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader


## Parameters

In [8]:
# Define hyperparameters
input_size = 1
hidden_size = 128
num_layers = 1
learning_rate = 0.001
num_epochs = 40
batch_size = 10

## Data

In [9]:

# Load the CSV data
data = pd.read_csv('data.csv')

# Define input and output
X1 = data['num1'].values
X2 = data['num2'].values
y = data['label'].values
data

,num1,num2,label
0,81,67,0
1,67,25,1
2,93,78,0
3,58,54,0
4,23,83,1
...,...,...,...
395,15,7,0
396,74,68,0
397,58,54,0
398,52,37,0


## Twin Rnn Model

In [14]:
# Define a custom LSTM model
class TwinLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers=1):
        super(TwinLSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size, hidden_size, num_layers)
        self.lstm2 = nn.LSTM(input_size, hidden_size, num_layers)
        self.fc = nn.Linear(hidden_size*2, 1)

    def forward(self, x1, x2):
        _, (hidden1, _) = self.lstm1(x1.unsqueeze(0))
        _, (hidden2, _) = self.lstm2(x2.unsqueeze(0))
        combined_hidden = torch.cat((hidden1, hidden2), dim=1)
        output = torch.sigmoid(self.fc(combined_hidden))
        ## 
        return output



In [13]:
X1_train, X1_test, X2_train, X2_test, y_train, y_test = train_test_split(X1, X2, y, test_size=0.2, random_state=42)

X1_train = torch.tensor(X1_train, dtype=torch.float32)
X2_train = torch.tensor(X2_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32) 
X1_test = torch.tensor(X1_test, dtype=torch.float32)
X2_test = torch.tensor(X2_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32) 


# Create a TensorDataset
train_dataset = TensorDataset(X1_train, X2_train, y_train)

# Define hyperparameters
batch_size = 20
num_epochs = 100
learning_rate = 0.001

# Create a DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


model = TwinLSTM(input_size, hidden_size, num_layers)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


## Training


In [12]:
# Training loop
for epoch in range(num_epochs):
    for x1, x2, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(x1, x2)
        loss = criterion(outputs.unsqueeze(1), labels.squeeze(1)) 
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}")

    with torch.no_grad():
        predicted = (outputs > 0.5).float()
        accuracy = (predicted == labels).float().mean()
        print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {accuracy.item()}")


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

## CheckSize

In [ ]:
print(outputs.size())
print(outputs.shape)

print(x1.size())
print(x2.size())
print(x1.shape)
print(x2.shape)

print(labels.shape)


## Evaluate

In [ ]:

# Testing the model
with torch.no_grad():
    outputs = model(X1_test.view(-1, 1, 1), X2_test.view(-1, 1, 1))
    predicted = (outputs.view(-1) > 0.5).float()
    accuracy = (predicted == y_test).float().mean()
    print(f'Accuracy: {accuracy.item()*100:.2f}%')